In [2]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm, sys
from bs4 import BeautifulSoup
from selenium import webdriver

In [42]:
driver = webdriver.Chrome()

In [43]:
def scrape_urls(url):
    driver.get(url)
    time.sleep(10)
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    try:
        link_num = int(soup.find('h2', id='query-summary').text.split()[0].replace(',', '')) # xxx URLs have been captured...
    except:
        print('No Result')
        return
    links = [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")] # make a list of URL
    for i in range ((link_num-1) // 50):
        driver.find_element_by_id('resultsUrl_next').click()
        time.sleep(2)
        html = driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
        links += [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")] # append URL in next page to list
    return links

In [44]:
links = scrape_urls('https://web.archive.org/web/*/https://www3.nhk.or.jp/nhkworld/th/news/*')

In [45]:
with open('nhkthailink.json', 'w') as f:
    json.dump(links, f, indent=4)

# scrape normal URL

In [7]:
year = 2019

for m in range(7,13):
    with open(f'linknormal{year}.txt', 'a') as f:
        for d in range(1, 32):
            m, d = str(m), str(d)
            if len(m) == 1:
                m = '0' + m
            if len(d) == 1:
                d = '0' + d
            driver.get(f'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/html/{year}{m}{d}/*')
            time.sleep(10)
            html = driver.page_source.encode('utf-8')
            soup = BeautifulSoup(html, 'html.parser')
            try:
                link_num = int(soup.find('h2', id='query-summary').text.split()[0]) # xxx URLs have been captured...
            except:
                continue
            links = [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")] # make a list of URL
            links = [link for link in links if '.html' in link] # only html
            for i in range ((link_num-1) // 50):
                driver.find_element_by_id('resultsUrl_next').click()
                time.sleep(2)
                links += [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")] # append URL in next page to list
                links = [link for link in links if '.html' in link]
            if links != []:
                f.write('\n'.join(links)+'\n') # append to file

    # sort
    id_exist = set(pd.read_json(f'nhkweb{year}.json').id)
    links = pd.read_csv(f'linknormal{year}.txt', header=None)
    links = [link for link in links[0] if link.split('.html')[0].split('/')[-1] not in id_exist]
    links = sorted(set(links))
    with open(f'linknormal{year}.txt', 'w') as f:
        f.write('\n'.join(links) + '\n')

In [9]:
# sort
id_exist = set(pd.read_json(f'nhkweb{year}.json').id)
links = pd.read_csv(f'linknormal{year}.txt', header=None)
links = [link for link in links[0] if link.split('.html')[0].split('/')[-1] not in id_exist]
links = sorted(set(links))
with open(f'linknormal{year}.txt', 'w') as f:
    f.write('\n'.join(links) + '\n')

In [12]:
# only one day
with open('linknormal.txt', 'a') as f:
    driver.get(f'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/html/20130127/k100/*')
    time.sleep(10)
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    try:
        link_num = int(soup.find('h2', id='query-summary').text.split()[0])
    except:
        sys.exit()
    links = [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")]
    links = [link for link in links if link.endswith('html') or 'html?utm' in link]  
    for i in range ((link_num-1) // 50):
        driver.find_element_by_id('resultsUrl_next').click()
        time.sleep(2)
        links += [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")]
        links = [link for link in links if link.endswith('html') or 'html?utm' in link]
    if links != []:
        f.write('\n'.join(links)+'\n')
                
with open('linknormal.txt', 'r') as f:
    id_exist = set(pd.read_json('nhkweb.json')['id'].tolist())
    links = f.read().strip().split('\n')
    links = [link for link in links if link.split('.html')[0].split('/')[-1] not in id_exist]
    links = sorted(set(links))
with open('linknormal.txt', 'w') as f:
    f.write('\n'.join(links) + '\n')

# scrape easy URL

In [8]:
with open('linkeasy.txt') as f:
    urls = f.read().strip().split('\n')
    ids = set([x.split('.html')[0].split('/')[-1] for x in urls])

id_exist = set(pd.read_json('nhkwebeasy.json')['id'].tolist())
print(len(ids), len(id_exist))
len(ids - id_exist)

1 5757


1

In [4]:
driver = webdriver.Firefox()

In [7]:
with open('linkeasy.txt', 'a') as f:
    for i in range(110, 120):
        driver.get(f'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k100{i}/*')
        time.sleep(10)
        html = driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
        try:
            link_num = int(soup.find('h2', id='query-summary').text.split()[0])
        except:
            continue
        links = [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")]
        links = [link for link in links if link.endswith('html') or 'html?utm' in link]  
        for i in range ((link_num-1) // 50):
            driver.find_element_by_id('resultsUrl_next').click()
            time.sleep(2)
            links += [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")]
            links = [link for link in links if link.endswith('html') or 'html?utm' in link]
        if links != []:
            f.write('\n'.join(links)+'\n')
            
with open('linkeasy.txt', 'r') as f:
    id_exist = set(pd.read_json('nhkwebeasy.json')['id'].tolist())
    links = f.read().strip().split('\n')
    links = [link for link in links if link.split('.html')[0].split('/')[-1] not in id_exist]
    links = sorted(set(links))
with open('linkeasy.txt', 'w') as f:
    f.write('\n'.join(links) + '\n')

In [ ]:
len(pd.read_json('nhk'))